In [46]:
import pandas as pd
import numpy as np
import os
import sys
data_dir = os.getcwd()+"/log_data/"
sys.argv = ["file",data_dir+"train.csv", data_dir+"test.csv", data_dir+"param_a.txt" ,data_dir+"outputfile.csv", data_dir+"weightfile.csv"]


In [71]:
def hotencode_train(data):
    train = pd.DataFrame()
    parents = pd.get_dummies(data[0], prefix="parents")
    has_nurs = pd.get_dummies(data[1], prefix="has_nurs")
    form = pd.get_dummies(data[2], prefix="form")
    children = pd.get_dummies(data[3], prefix="children")
    housing = pd.get_dummies(data[4], prefix="housing")
    finance = pd.get_dummies(data[5], prefix="finance")
    social = pd.get_dummies(data[6], prefix="social")
    health = pd.get_dummies(data[7], prefix="health")
    classDistribution = pd.get_dummies(data[8], prefix="class")
    train = pd.concat([parents, has_nurs, form, children, housing,
                    finance, social, health, classDistribution], axis=1)
    cols = ['parents_usual', 'parents_pretentious', 'parents_great_pret', 'has_nurs_proper', 'has_nurs_less_proper', 'has_nurs_improper',
            'has_nurs_critical', 'has_nurs_very_crit',
            'form_complete', 'form_completed', 'form_incomplete', 'form_foster',
            'children_1', 'children_2', 'children_3', 'children_more',
            'housing_convenient', 'housing_less_conv', 'housing_critical',
            'finance_convenient', 'finance_inconv',
            'social_nonprob', 'social_slightly_prob', 'social_problematic',
            'health_recommended', 'health_priority', 'health_not_recom',
            'class_not_recom', 'class_recommend', 'class_very_recom', 'class_priority', 'class_spec_prior']

    return train[pd.Index(cols)]

def hotencode_test(data):
    train = pd.DataFrame()
    parents = pd.get_dummies(data[0], prefix="parents")
    has_nurs = pd.get_dummies(data[1], prefix="has_nurs")
    form = pd.get_dummies(data[2], prefix="form")
    children = pd.get_dummies(data[3], prefix="children")
    housing = pd.get_dummies(data[4], prefix="housing")
    finance = pd.get_dummies(data[5], prefix="finance")
    social = pd.get_dummies(data[6], prefix="social")
    health = pd.get_dummies(data[7], prefix="health")
    train = pd.concat([parents, has_nurs, form, children, housing,
                    finance, social, health], axis=1)
    cols = ['parents_usual', 'parents_pretentious', 'parents_great_pret', 'has_nurs_proper', 'has_nurs_less_proper', 'has_nurs_improper',
            'has_nurs_critical', 'has_nurs_very_crit',
            'form_complete', 'form_completed', 'form_incomplete', 'form_foster',
            'children_1', 'children_2', 'children_3', 'children_more',
            'housing_convenient', 'housing_less_conv', 'housing_critical',
            'finance_convenient', 'finance_inconv',
            'social_nonprob', 'social_slightly_prob', 'social_problematic',
            'health_recommended', 'health_priority', 'health_not_recom']
    return train[pd.Index(cols)]

In [72]:
train_data = pd.read_csv(sys.argv[1], header=None)
test_data = pd.read_csv(sys.argv[2], header=None)
train = hotencode_train(train_data)
test = hotencode_test(test_data)
classes = train.iloc[:, -5:].values
features = train.iloc[:, :-5].values
features = np.c_[np.ones(len(train)), features]
X_train = features[:5400, :]
Y_train = classes[:5400, :]
X_test = features[5400:, :]
Y_test = classes[5400:, :]
print(X_train.shape, Y_train.shape,X_test.shape)


(5400, 28) (5400, 5) (600, 28)


In [73]:
def gradient(X,Y,W):
    XW = np.exp(np.matmul(X, W))
    denom = np.sum(XW, axis=1)
    Y_predict = np.divide(XW, denom.reshape(X.shape[0], 1))
    return np.matmul(X.transpose(), Y - Y_predict)/X.shape[0]
def cost(W,X,Y):
    XW = np.exp(np.matmul(X,W))
    logTerm = np.log(np.sum(XW,axis=1))
    weightedTerm = np.sum(np.multiply(np.matmul(Y,W.T),X),axis=1)
    error = np.sum(logTerm-weightedTerm)/X.shape[0]
    return error

In [74]:
alpha = [0.00001,0.0001,0.001,0.01, .1, 1,10,2500,25000,30000]
# alpha = [0.1,.02]
batch = [10,20,30]
lamIndex=0
batchIndex=0
a = np.zeros((len(alpha),len(batch)))
f1= 0
for m in range(0,len(batch)):
    for n in range(0,len(alpha)):
        X_train = np.copy(features[:5400, :])
        Y_train = np.copy(classes[:5400, :])
        X_test = np.copy(features[5400:, :])
        Y_test = np.copy(classes[5400:, :])
        w_initial = np.zeros(28*5).reshape(28, 5)
        # w_initial = (w_initial.T - w_initial.T[0]).T
        costAr = []
        k = batch[m]
        l = X_train.shape[0]
        costAr.append(0)
        for j in range(1, 10000):
            for i in range(0,k):
                grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
                w_initial = w_initial+0.01*(grad-alpha[n]*w_initial)
                # w_initial = (w_initial.T - w_initial.T[0]).T
            c = cost(w_initial,X_train,Y_train)
            print((j,c), end="\r", flush=True)
            costAr.append(c)
            if(costAr[j-1]-costAr[j]<.00001 and j>10):
                print(costAr[j],costAr[j-1])
                break
        print(j)
        w_initial = (w_initial.T - w_initial.T[0]).T
        WmulX = np.exp(np.matmul(X_test, w_initial))
        denom = np.sum(WmulX, axis=1)
        Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
        b = np.zeros_like(Y_predict)
        b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
        Y_predict = b
        accuracy = np.trace(np.matmul(Y_predict, Y_test.T))/len(X_test)
        Y_predict[:,0] =1*Y_predict[:,0]
        Y_predict[:,1] =2*Y_predict[:,1]
        Y_predict[:,2] =3*Y_predict[:,2]
        Y_predict[:,3] =4*Y_predict[:,3]
        Y_predict[:,4] =5*Y_predict[:,4]
        Y_predict = np.sum(Y_predict,axis=1).tolist()
        Y_test[:,0] =1*Y_test[:,0]
        Y_test[:,1] =2*Y_test[:,1]
        Y_test[:,2] =3*Y_test[:,2]
        Y_test[:,3] =4*Y_test[:,3]
        Y_test[:,4] =5*Y_test[:,4]
        Y_test = np.sum(Y_test,axis=1).tolist()
        mat = np.zeros((5,5))
        f1Score = []
        for i in range(0,len(Y_predict)):
            mat[int(Y_predict[i])-1][Y_test[i]-1]=mat[int(Y_predict[i])-1][Y_test[i]-1]+1
        for i in range(len(mat)):
            precision = mat[i,i]/np.sum(mat[:,i])
            recall = mat[i,i]/np.sum(mat[i,:])
            f1Score.append([mat[i,i],np.sum(mat[:,i])-mat[i,i],np.sum(mat[i,:])-mat[i,i],precision,recall,precision*recall/(precision+recall)*100])  
        F1Score = pd.DataFrame([],columns=["TP","FP","FN","precision","recall","F1_Score"])
        F1Score.loc["class_not_recom"] = f1Score[0]
        F1Score.loc["class_recommend"] = f1Score[1]
        F1Score.loc["class_very_recom"] = f1Score[2]
        F1Score.loc["class_priority"] = f1Score[3]
        F1Score.loc["class_spec_prior"] = f1Score[4]
        micro_average_precision = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,1].values))
        micro_average_recall = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,2].values))
        micro_F1_score = micro_average_precision*micro_average_recall/(micro_average_precision+micro_average_recall)
        print("micro_F1_score= ",2*micro_F1_score,m,accuracy)
        if(f1<2*micro_F1_score and micro_F1_score==micro_F1_score):
            print("updated")
            f1= 2*micro_F1_score
            lamIndex=n
            batchIndex = m
        a[n][m]=2*micro_F1_score
print(a)

0.21645523518424617 0.2164652335432906
3486
micro_F1_score=  0.9133333333333333 0 0.9133333333333333
updated


file:57: RuntimeWarning: invalid value encountered in double_scalars
file:58: RuntimeWarning: invalid value encountered in double_scalars


0.21820424575790184 0.21821424189997174
3444
micro_F1_score=  0.9133333333333333 0 0.9133333333333333
0.23594372376460762 0.23595371781976315
3002
micro_F1_score=  0.915 0 0.915
updated
0.3468926537423164 0.3469026420469391
1350
micro_F1_score=  0.9133333333333333 0 0.9133333333333333
0.7007174728398741 0.7007273885027991
369
micro_F1_score=  0.9016666666666667 0 0.9016666666666666
1.2827472247041707 1.282756842579927
67
micro_F1_score=  0.8666666666666667 0 0.8666666666666667
1.5577928605920721 1.5577933854063788
11
micro_F1_score=  0.7983333333333333 0 0.7983333333333333


file:4: RuntimeWarning: invalid value encountered in true_divide


99999, nan)
micro_F1_score=  0.31 0 0.31
99999, nan)
micro_F1_score=  0.31 0 0.31
99999, nan)
micro_F1_score=  0.31 0 0.31
0.2037589439425205 0.2037689427238259
2641
micro_F1_score=  0.9183333333333333 1 0.9183333333333333
updated
0.2057895984029242 0.20579959460863867
2600
micro_F1_score=  0.9183333333333333 1 0.9183333333333333
0.22672791960685337 0.22673791688788125
2149
micro_F1_score=  0.915 1 0.915
0.34469179623188684 0.3447017833201655
829
micro_F1_score=  0.9133333333333333 1 0.9133333333333333
0.7004151717024818 0.7004251209294677
206
micro_F1_score=  0.9016666666666667 1 0.9016666666666666
1.2826947313220114 1.282704048154966
37
micro_F1_score=  0.8633333333333333 1 0.8633333333333333
1.5577608047165503 1.557760804728818
11
micro_F1_score=  0.7783333333333333 1 0.7783333333333333
99999, nan)
micro_F1_score=  0.31 1 0.31
99999, nan)
micro_F1_score=  0.31 1 0.31


KeyboardInterrupt: 

In [14]:
alpha = [0.000005,0.00001,0.00005,0.0001, .001, .01,0.1,1,10]
batch = [10,1]
a = np.zeros((len(alpha),len(batch)))

In [78]:
alpha = [0.1,.02]
batch = [1]
lamIndex=0
batchIndex=0
a = np.zeros((len(alpha),len(batch)))
f1= 0
# for m in range(0,len(batch)):
# for n in range(0,len(alpha)):
X_train = np.copy(features[:5400, :])
Y_train = np.copy(classes[:5400, :])
X_test = np.copy(features[5400:, :])
Y_test = np.copy(classes[5400:, :])
w_initial = np.zeros(28*5).reshape(28, 5)
# w_initial = (w_initial.T - w_initial.T[0]).T
costAr = []
k = batch[0]
l = X_train.shape[0]
costAr.append(0)
for j in range(1, 10000):
    for i in range(0,k):
        grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
        w_initial = w_initial+0.01*(grad-alpha[0]*w_initial)
        # w_initial = (w_initial.T - w_initial.T[0]).T
    c = cost(w_initial,X_train,Y_train)
    print((j,c), end="\r", flush=True)
    costAr.append(c)
    if(costAr[j-1]-costAr[j]<.00001 and j>10):
        print(costAr[j],costAr[j-1])
        break
print(j)


0.7059994031552381 0.7060093910528225
2314


In [66]:
classes

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 4, 0],
       [0, 0, 0, 4, 0],
       [0, 0, 0, 0, 5]], dtype=uint8)

In [102]:
X_train = np.copy(features[:5400, :])
Y_train = np.copy(classes[:5400, :])
X_test = np.copy(features[5400:, :])
Y_test = np.copy(classes[5400:, :])

In [104]:
cl = np.sum(Y_test,axis=0)

In [95]:
WmulX = np.exp(np.matmul(X_test, w_initial))
denom = np.sum(WmulX, axis=1)
Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
b = np.zeros_like(Y_predict)
b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
Y_predict = b
accuracy = np.trace(np.matmul(Y_predict, Y_test.T))/len(X_test)
Y_predict[:,0] =1*Y_predict[:,0]
Y_predict[:,1] =2*Y_predict[:,1]
Y_predict[:,2] =3*Y_predict[:,2]
Y_predict[:,3] =4*Y_predict[:,3]
Y_predict[:,4] =5*Y_predict[:,4]
Y_predict = np.sum(Y_predict,axis=1).tolist()
Y_test[:,0] =1*Y_test[:,0]
Y_test[:,1] =2*Y_test[:,1]
Y_test[:,2] =3*Y_test[:,2]
Y_test[:,3] =4*Y_test[:,3]
Y_test[:,4] =5*Y_test[:,4]
Y_test = np.sum(Y_test,axis=1).tolist()
mat = np.zeros((5,5))
#     if(f1<micro_F1_score):
#         print("updated")
#         lamIndex=n
#         batchIndex = 0
#     a[0][m]=micro_F1_score
# print(a)

In [97]:
f1Score = []
for i in range(0,len(Y_predict)):
    mat[int(Y_predict[i])-1][Y_test[i]-1]=mat[int(Y_predict[i])-1][Y_test[i]-1]+1
mat

array([[186.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  15., 200.,  29.],
       [  0.,   0.,   0.,  15., 155.]])

In [98]:

for i in range(len(mat)):
    TP = mat[i,i]
    FP = np.sum(mat[:,i])-mat[i,i]
    FN = np.sum(mat[i,:])-mat[i,i]
    if(TP==0 and FP==0 and FN==0):
        precision=100
        recall=100
        F1_score=100
    elif(TP==0 and (FP==0 or FN==0)):
        precision=0
        recall=0
        F1_score=0
    else:
        precision = mat[i,i]/np.sum(mat[:,i])
        recall = mat[i,i]/np.sum(mat[i,:])
        F1_score = 2*precision*recall/(precision+recall)*100
    f1Score.append([TP,FP,FN,precision,recall,F1_score])  

# micro_average_precision = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,1].values))
# micro_average_recall = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,2].values))
# micro_F1_score = micro_average_precision*micro_average_recall/(micro_average_precision+micro_average_recall)
# print("micro_F1_score= ",2*micro_F1_score,accuracy)


In [110]:
micro_average_precision = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,1].values))
micro_average_recall = np.sum(F1Score.iloc[:,0].values)/(np.sum(F1Score.iloc[:,0].values)+np.sum(F1Score.iloc[:,2].values))
micro_F1_score = micro_average_precision*micro_average_recall/(micro_average_precision+micro_average_recall)
print("micro_F1_score= ",2*micro_F1_score,accuracy)


micro_F1_score=  0.9016666666666667 0.9016666666666666


In [105]:
F1Score = pd.DataFrame([],columns=["TP","FP","FN","precision","recall","F1_Score"])
F1Score.loc["class_not_recom"] = f1Score[0]
F1Score.loc["class_recommend"] = f1Score[1]
F1Score.loc["class_very_recom"] = f1Score[2]
F1Score.loc["class_priority"] = f1Score[3]
F1Score.loc["class_spec_prior"] = f1Score[4]
Weighted_F1_score = np.dot(cl,F1Score.iloc[:,5].values)/np.sum(cl)

,TP,FP,FN,precision,recall,F1_Score
class_not_recom,186.0,0.0,0.0,1.000000,1.000000,100.000000
class_recommend,0.0,0.0,0.0,100.000000,100.000000,100.000000
class_very_recom,0.0,15.0,0.0,0.000000,0.000000,0.000000
class_priority,200.0,15.0,44.0,0.930233,0.819672,87.145969
class_spec_prior,155.0,29.0,15.0,0.842391,0.911765,87.570621


In [109]:
Weighted_F1_score = np.dot(cl,F1Score.iloc[:,5].values)/np.sum(cl)

89.08229632091381

In [68]:
mat

array([[186.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  15., 200.,  29.],
       [  0.,   0.,   0.,  15., 155.]])

In [22]:
costOverall = []
a=np.zeros(len(alpha))
for n in range(0,len(alpha)):
    w_initial = np.ones(28*5).reshape(28, 5)
    w_initial = (w_initial.T - w_initial.T[0]).T
    costAr = []
    lr = .01
    costAr.append(0)
    for j in range(1, 10000):
        grad = gradient(X_train,Y_train,w_initial)
        w_initial = w_initial+lr*(grad-alpha[n]*w_initial)
        c = cost(w_initial,X_train,Y_train)
        print((j,c), end="\r", flush=True)
        costAr.append(c)
        if(costAr[j-1]-costAr[j]<.00001 and j>10):
            print(costAr[j],costAr[j-1])
            break
    WmulX = np.exp(np.matmul(X_test, w_initial))
    denom = np.sum(WmulX, axis=1)
    Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
    b = np.zeros_like(Y_predict)
    b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
    Y_predict = b
    accuracy = np.trace(np.matmul(Y_predict, Y_test.T))/len(X_test)
    print(accuracy)
    a[n]=accuracy        
    costOverall.append(costAr)  
print(a)
    

0.297173260700423270042327) 0.2971832603049492
0.9133333333333333
0.2972144106680319 0.29722441005367906
0.9133333333333333
0.2975536255440337 0.2975636252465655
0.9133333333333333
0.2979751828725619 0.2979851824334659
0.9133333333333333
0.30557438741251436 0.3055843852475009
0.915
0.374876909888105 0.3748869094203459
0.9116666666666666
0.7059994031552381 0.7060093910528225
0.9016666666666666
1.2835392340571263 1.2835492181785806
0.8666666666666667
1.5578892523086028 1.5578987171693646
0.81
[0.91333333 0.91333333 0.91333333 0.91333333 0.915      0.91166667
 0.90166667 0.86666667 0.81      ]


In [ ]:
for m in range(0,len(batch)):
    for n in range(0,len(alpha)):
        w_initial = np.zeros(28*5).reshape(28, 5)
        # w_initial = (w_initial.T - w_initial.T[0]).T
        costAr = []
        k = batch[m]
        l = X_train.shape[0]
        costAr.append(0)
        for j in range(1, 10000):
            for i in range(0,k):
                grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
                w_initial = w_initial+0.01*(grad-alpha[n]*w_initial)
                # w_initial = (w_initial.T - w_initial.T[0]).T
            c = cost(w_initial,X_train,Y_train)
            print((j,c), end="\r", flush=True)
            costAr.append(c)
            if(costAr[j-1]-costAr[j]<.00001 and j>10):
                print(costAr[j],costAr[j-1])
                break
        print(j)
        w_initial = (w_initial.T - w_initial.T[0]).T
        WmulX = np.exp(np.matmul(X_test, w_initial))
        denom = np.sum(WmulX, axis=1)
        Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
        b = np.zeros_like(Y_predict)
        b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
        Y_predict = b
        accuracy = np.trace(np.matmul(Y_predict, Y_test.T))/len(X_test)
        print(m,accuracy)
        a[n][m]=accuracy
print(a)
X_train = features[:, :]
X_train[:,21]=0
Y_train = classes[:, :]
print(X_train.shape,Y_train.shape)
lamIndex, batchIndex =  np.where(a==np.max(a))[0][0], np.where(a==np.max(a))[1][0]
w_initial = np.ones(28*5).reshape(28, 5)
w_initial = (w_initial.T - w_initial.T[0]).T
costAr = []
k = batch[batchIndex]
l = X_train.shape[0]
costAr.append(0)
for j in range(1, 10000):
    for i in range(0,k):
        grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
        w_initial = w_initial+0.01*(grad-alpha[lamIndex]*w_initial)
        w_initial = (w_initial.T - w_initial.T[0]).T
    c = cost(w_initial,X_train,Y_train)
    print((j,c), end="\r", flush=True)
    costAr.append(c)
    if(costAr[j-1]-costAr[j]<.00001 and j>10):
        print(costAr[j],costAr[j-1])
        break
print(j)
features = test.iloc[:, :].values
features = np.c_[np.ones(len(test)), features]
X_test = features[:, :]
X_test[:,21]=0
WmulX = np.exp(np.matmul(X_test, w_initial))
denom = np.sum(WmulX, axis=1)
Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
b = np.zeros_like(Y_predict)
b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
Y_predict = b
Y_predict[:, 0] = 1*Y_predict[:, 0]
Y_predict[:, 1] = 2*Y_predict[:, 1]
Y_predict[:, 2] = 3*Y_predict[:, 2]
Y_predict[:, 3] = 4*Y_predict[:, 3]
Y_predict[:, 4] = 5*Y_predict[:, 4]
Y_predict = np.sum(Y_predict, axis=1).tolist()
predict = []
for i in range(0, len(Y_predict)):
    if(Y_predict[i] == 1):
        predict.append("not_recom")
    elif(Y_predict[i] == 2):
        predict.append("recommend")
    elif(Y_predict[i] == 3):
        predict.append("very_recom")
    elif(Y_predict[i] == 4):
        predict.append("priority")
    elif(Y_predict[i] == 5):
        predict.append("spec_prior")
Y_predict = pd.DataFrame(predict)
Y_predict.to_csv(sys.argv[3], header=False, index=False)
w_initial = pd.DataFrame(w_initial)
w_initial.to_csv(sys.argv[4], header=False, index=False)